---
title: C++ Primer Plus ch18.2 Visiting with the new C++ standard Move semantics and Rvalue reference
tags: 小书匠,Books-C++PrimerPlus,c++,move,rvalue
grammar_cjkRuby: true
renderNumberedHeading: true
---
    
[toc]

# C++ Primer Plus ch18.2 Visiting with the new C++ standard Move semantics and Rvalue reference

In [14]:
%%file useless.cpp
#include <iostream>
using namespace std;

class Useless
{
    private:
        int n;
        char* pc;
        static int ct;
        void ShowObject() const;
    public:
        Useless();
        explicit Useless(int k);
        Useless(int k, char ch);
        Useless(const Useless& f); // copy constructor
        Useless(Useless&& f);      // move constructor
        ~Useless();
        Useless operator+(const Useless& f) const;
        void ShowData() const;
};

int Useless::ct = 0;
Useless::Useless()
{
    ++ct;
    n = 0;
    pc = NULL;
    cout << "default constructor called; number of objects: " << ct << endl;
    ShowObject();
}

Useless::Useless(int k): n(k)
{
    ++ct;
    cout << "int constructor called; number of objects: " << ct << endl;
    pc = new char[n];
    ShowObject();
}

Useless::Useless(int k, char ch): n(k)
{
    ++ct;
    cout << "int, char constructor called; number of objects: " << ct << endl;
    pc = new char[n];
    for (int i=0; i<n; i++)
        pc[i] = ch;
    ShowObject();
}

Useless::Useless(const Useless& f): n(f.n)
{
    ++ct;
    cout << "copy constructor called; number of objects: " << ct << endl;
    pc = new char[n];
    for (int i=0; i<n; i++)
        pc[i] = f.pc[i];
    ShowObject();
}

Useless::Useless(Useless&& f): n(f.n)
{
    ++ct;
    cout << "move constructor called; number of objects: " << ct << endl;
    pc = f.pc;
    f.pc = NULL;
    f.n = 0;
    ShowObject();
}

Useless::~Useless()
{
    cout << "destructor called; object left: " << --ct << endl;
    cout << "delete object:\n";
    ShowObject();
    delete [] pc;
}

Useless Useless::operator+(const Useless& f) const
{
    cout << "Entering operator+()\n";
    Useless temp = Useless(n + f.n); // int constructor
    for (int i=0; i<n; i++)
    {
        temp.pc[i] = pc[i];
    }
    for (int i=n; i<temp.n; i++)
        temp.pc[i] = f.pc[i - n];
    cout << "temp object:\n";
    cout << "Leaving operator+()\n";
    return temp; // return a rvalue
}

void Useless::ShowObject() const 
{
    cout << "Number of elements: " << n
         << " Data address: " << (void*) pc << endl;
}

void Useless::ShowData() const 
{
    if (n == 0)
        cout << "(object empty)";
    else
        for (int i=0; i<n; i++)
            cout << pc[i];
    cout << endl;
}

int main()
{
    {
        Useless one(10, 'x'); // call int, char constructor
        Useless two = one; // call copy constructor
        Useless three(20, 'o'); // call int, char constructor
        Useless four(one + three); // call move constructor
        cout << "object one:";
        one.ShowData();
        cout << "object two:";
        two.ShowData();
        cout << "object three:";
        three.ShowData();
        cout << "object four:";
        four.ShowData();
    }
}

Overwriting useless.cpp


In [15]:
!g++ useless.cpp -std=c++11 -o useless && ./useless

int, char constructor called; number of objects: 1
Number of elements: 10 Data address: 0x55985877fec0
copy constructor called; number of objects: 2
Number of elements: 10 Data address: 0x55985877fee0
int, char constructor called; number of objects: 3
Number of elements: 20 Data address: 0x55985877ff00
Entering operator+()
int constructor called; number of objects: 4
Number of elements: 30 Data address: 0x55985877ff20
temp object:
Leaving operator+()
object one:xxxxxxxxxx
object two:xxxxxxxxxx
object three:oooooooooooooooooooo
object four:xxxxxxxxxxoooooooooooooooooooo
destructor called; object left: 3
delete object:
Number of elements: 30 Data address: 0x55985877ff20
destructor called; object left: 2
delete object:
Number of elements: 20 Data address: 0x55985877ff00
destructor called; object left: 1
delete object:
Number of elements: 10 Data address: 0x55985877fee0
destructor called; object left: 0
delete object:
Number of elements: 10 Data address: 0x55985877fec0


The interesting thing is that if you compile the code with g++ and run it, no move constructor is called. Because the compiler move the result of `one+three` into `four` automatically, even without move constructor.

However, if the code is compiled using other compiler, move constructor may called.

# References